In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import os
import statistics
import math

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns', None)

In [4]:
Filepath_Roku_Viewership = 'C:/Users/sortiz/Box/Digital Networks/REPORTING + ANALYTICS/Linear/Roku/Roku Viewership/Weekly Content Data/'

In [5]:
# Variables

Channel = 'RetroCrush'
Weeks_list = ['Week 2023_04_02_2023_03_27','Week 2023_04_09_2023_04_03','Week 2023_04_16_2023_04_10']
Weeks_list_Num = list(range(len(Weeks_list)))
Date = Weeks_list[-1][5:15]

# HOV/Streams per Day (Channel Level)

In [6]:
L_1=[]

for Week in Weeks_list:
    L_1 = L_1 + [Filepath_Roku_Viewership + Channel + '/' + Week + '/' + 'streaming_hours_and_stream_starts_(channel_level).csv']
    
View_CHN = pd.concat(pd.read_csv(file) for file in L_1)    

In [7]:
View_CHN.dropna(inplace=True)
View_CHN.drop(columns=View_CHN.columns[0], axis=1,  inplace=True)
View_CHN.columns = ['Date','Sessions','HOV']

View_CHN['Sessions'] = View_CHN['Sessions'].str.replace(',','')
View_CHN['HOV'] = View_CHN['HOV'].str.replace(',','')

View_CHN['Date'] = pd.to_datetime(View_CHN['Date'])
View_CHN['Sessions'] = View_CHN['Sessions'].astype('float')
View_CHN['HOV'] = View_CHN['HOV'].astype('float')

View_CHN['Minutes Viewed'] = View_CHN['HOV'] * 60
View_CHN = View_CHN.drop_duplicates(keep = 'last', subset='Date')

In [8]:
View_CHN.head()

,Date,Sessions,HOV,Minutes Viewed
1,2023-03-13,29774.0,4720.98,283258.8
2,2023-03-14,28552.0,4783.05,286983.0
3,2023-03-15,27180.0,4660.82,279649.2
4,2023-03-16,24550.0,4486.28,269176.8
5,2023-03-17,26629.0,4632.58,277954.8


# HOV/Streams per Day (Title Level)

In [9]:
L_2=[]

for Week in Weeks_list:
    L_2 = L_2 + [Filepath_Roku_Viewership + Channel + '/' + Week + '/' + 'streaming_hours_and_stream_starts_(by_title).csv']   

In [10]:
def clean_title_table(data):
    VT = pd.read_csv(data)
    VT = VT.iloc[:,1:]
    VT = VT.melt(id_vars=['Date Dim'], var_name=['Date'], value_name='Value').dropna()
    VT_HOV = VT.loc[VT['Date'].str.contains('.1', regex=False),:]
    VT_STR = VT.loc[~VT['Date'].str.contains('.1', regex=False),:]
    VT_HOV = VT_HOV.loc[VT_HOV['Value']!='Streaming Hours Sum',:]
    VT_STR = VT_STR.loc[VT_STR['Value']!='Stream Start Event Count',:]
    VT_HOV['Date'] = VT_HOV['Date'].str.replace('.1','', regex=False)
    VT_STR['Value'] = VT_STR['Value'].str.replace(',','', regex=False)
    VT_HOV['Value'] = VT_HOV['Value'].str.replace(',','', regex=False)
    VT_HOV.rename(columns={'Date Dim':'Title','Value':'HOV'}, inplace=True)
    VT_STR.rename(columns={'Date Dim':'Title','Value':'Sessions'}, inplace=True)
    VT_HOV['HOV'] = VT_HOV['HOV'].astype('float')
    VT_STR['Sessions'] = VT_STR['Sessions'].astype('float')
    VT_HOV['Minutes Viewed'] = VT_HOV['HOV']*60
    VT_HOV_STR = pd.merge(VT_HOV,VT_STR, on=['Title','Date'], how='left')
    return VT_HOV_STR.copy()

In [11]:
View_Title = pd.DataFrame()

for data in L_2:
    View_Title = View_Title.append(clean_title_table(data))

In [12]:
View_Title = View_Title.drop_duplicates(keep='last', subset=['Title','Date'])

In [13]:
View_Title.head()

,Title,Date,HOV,Minutes Viewed,Sessions
0,The Twelve Kingdoms,2023-03-13,594.47,35668.2,6214.0
1,Knight Hunters Eternity,2023-03-13,464.54,27872.4,4764.0
2,Virtua Fighter,2023-03-13,525.09,31505.4,3260.0
3,Street Fighter - The Animated,2023-03-13,373.89,22433.4,2639.0
4,Night on the Galactic Railroad (English Dubbed),2023-03-13,369.14,22148.4,2468.0


# Dayparting Channel Level (Last complete week)

In [14]:
L_3=[]

for Week in Weeks_list:
    L_3 = L_3 + [Filepath_Roku_Viewership + Channel + '/' + Week + '/' + 'Dayparting (Channel Level) (Last Complete Week).csv']  

In [15]:
def clean_Channel_Daypart(data,weekN):
    DayP_CHN = pd.read_csv(data)
    DayP_CHN['Week'] = Weeks_list[weekN]
    DayP_CHN = DayP_CHN.iloc[:,1:]
    DayP_CHN.columns = DayP_CHN.iloc[0]
    DayP_CHN = DayP_CHN[1:]
    DayP_CHN.columns = ['Start Hour','HOV','Sessions','Week']
    DayP_CHN['HOV'] = DayP_CHN['HOV'].str.replace(',','', regex=False)
    DayP_CHN['Sessions'] = DayP_CHN['Sessions'].str.replace(',','', regex=False)
    DayP_CHN['HOV'] = DayP_CHN['HOV'].astype('float')
    DayP_CHN['Sessions'] = DayP_CHN['Sessions'].astype('float')
    DayP_CHN['Minutes Viewed'] = DayP_CHN['HOV']*60
    DayP_CHN['Week'] = DayP_CHN['Week'].str.replace('Week ','')
    DayP_CHN['Week'] = DayP_CHN['Week'].str.replace('_202','/202')
    DayP_CHN['Week'] = DayP_CHN['Week'].str.replace('_','-')
    return DayP_CHN.copy()

In [16]:
Daypart_Channel = pd.DataFrame()

for data,weekN in zip(L_3,Weeks_list_Num):
    Daypart_Channel = Daypart_Channel.append(clean_Channel_Daypart(data, weekN))

In [17]:
#Adding Numbers to Start Hour

Condition =[
    (Daypart_Channel['Start Hour']== '2am EST'),(Daypart_Channel['Start Hour']== '3am EST'),(Daypart_Channel['Start Hour']== '4am EST'),
    (Daypart_Channel['Start Hour']== '5am EST'),(Daypart_Channel['Start Hour']== '6am EST'),(Daypart_Channel['Start Hour']== '7am EST'),
    (Daypart_Channel['Start Hour']== '8am EST'),(Daypart_Channel['Start Hour']== '9am EST'),(Daypart_Channel['Start Hour']== '10am EST'),
    (Daypart_Channel['Start Hour']== '11am EST'),(Daypart_Channel['Start Hour']== '12pm EST'),(Daypart_Channel['Start Hour']== '1pm EST'),
    (Daypart_Channel['Start Hour']== '2pm EST'),(Daypart_Channel['Start Hour']== '3pm EST'),(Daypart_Channel['Start Hour']== '4pm EST'),
    (Daypart_Channel['Start Hour']== '5pm EST'),(Daypart_Channel['Start Hour']== '6pm EST'),(Daypart_Channel['Start Hour']== '7pm EST'),
    (Daypart_Channel['Start Hour']== '8pm EST'),(Daypart_Channel['Start Hour']== '9pm EST'),(Daypart_Channel['Start Hour']== '10pm EST'),
    (Daypart_Channel['Start Hour']== '11pm EST'),(Daypart_Channel['Start Hour']== '12am EST'),(Daypart_Channel['Start Hour']== '1am EST')
]
    

Num_Hour = ['2. 2am EST', '3. 3am EST', '4. 4am EST',
           '5. 5am EST', '6. 6am EST', '7. 7am EST', 
           '8. 8am EST', '9. 9am EST', '10. 10am EST', 
           '11. 11am EST','12. 12pm EST', '13. 1pm EST', 
           '14. 2pm EST', '15. 3pm EST', '16. 4pm EST',
           '17. 5pm EST', '18. 6pm EST', '19. 7pm EST', 
           '20. 8pm EST', '21. 9pm EST', '22. 10pm EST',
           '23. 11pm EST','0. 12am EST', '1. 1am EST']

Daypart_Channel['Start Hour (Num)']= np.select(Condition, Num_Hour, default= np.nan)

In [18]:
Daypart_Channel = Daypart_Channel.loc[:,('Week','Start Hour (Num)','HOV','Minutes Viewed','Sessions')]

In [19]:
Daypart_Channel.head()

,Week,Start Hour (Num),HOV,Minutes Viewed,Sessions
1,2023-04-02/2023-03-27,19. 7pm EST,1177.80,70668.0,14594.0
2,2023-04-02/2023-03-27,20. 8pm EST,1990.12,119407.2,15143.0
3,2023-04-02/2023-03-27,21. 9pm EST,1341.99,80519.4,14418.0
4,2023-04-02/2023-03-27,22. 10pm EST,753.38,45202.8,12631.0
5,2023-04-02/2023-03-27,23. 11pm EST,1981.00,118860.0,9991.0


# Dayparting Title Level (Last complete week)

In [20]:
L_4=[]

for Week in Weeks_list:
    L_4 = L_4 + [Filepath_Roku_Viewership + Channel + '/' + Week + '/' + 'Dayparting (By Title) (Last Complete Week).csv']  

In [21]:
def Clean_Title_Daypart(Data,weekN):
    DayP_TIT = pd.read_csv(data)
    DayP_TIT = DayP_TIT.iloc[:,1:]
    DayP_TIT = DayP_TIT.melt(id_vars=['Secondary Movie/Series Title'], var_name=['Title'], value_name='Value').dropna()
    DayP_HOV = DayP_TIT.loc[~DayP_TIT['Title'].str.contains('.1', regex=False),:]
    DayP_STR = DayP_TIT.loc[DayP_TIT['Title'].str.contains('.1', regex=False),:]
    DayP_HOV = DayP_HOV.loc[DayP_HOV['Value']!='Streaming Hours Sum',:]
    DayP_STR = DayP_STR.loc[DayP_STR['Value']!='Stream Start Event Count',:]
    DayP_STR['Title'] = DayP_STR['Title'].str.replace('.1','', regex=False)
    DayP_STR['Value'] = DayP_STR['Value'].str.replace(',','', regex=False)
    DayP_HOV['Value'] = DayP_HOV['Value'].str.replace(',','', regex=False)
    DayP_HOV.rename(columns={'Secondary Movie/Series Title':'Start Hour','Value':'HOV'}, inplace=True)
    DayP_STR.rename(columns={'Secondary Movie/Series Title':'Start Hour','Value':'Sessions'}, inplace=True)
    DayP_HOV['HOV'] = DayP_HOV['HOV'].astype('float')
    DayP_STR['Sessions'] = DayP_STR['Sessions'].astype('float')
    DayP_HOV['Minutes Viewed'] = DayP_HOV['HOV']*60

    DayP_HOV['Week'] = Weeks_list[weekN]
    DayP_HOV['Week'] = DayP_HOV['Week'].str.replace('Week ','')
    DayP_HOV['Week'] = DayP_HOV['Week'].str.replace('_202','/202')
    DayP_HOV['Week'] = DayP_HOV['Week'].str.replace('_','-')

    DayP_STR['Week'] = Weeks_list[weekN]
    DayP_STR['Week'] = DayP_STR['Week'].str.replace('Week ','')
    DayP_STR['Week'] = DayP_STR['Week'].str.replace('_202','/202')
    DayP_STR['Week'] = DayP_STR['Week'].str.replace('_','-')

    DayP_HOV_STR = pd.merge(DayP_HOV,DayP_STR, on=['Week','Title','Start Hour'], how='left')
    return DayP_HOV_STR.copy()

In [22]:
Daypart_Title = pd.DataFrame()

for data,weekN in zip(L_4,Weeks_list_Num):
    Daypart_Title = Daypart_Title.append(Clean_Title_Daypart(data, weekN))

In [23]:
Daypart_Title['Session Duration'] = Daypart_Title['Minutes Viewed']/Daypart_Title['Sessions'] 

In [24]:
Condition =[
    (Daypart_Title['Start Hour']== '2am EST') | (Daypart_Title['Start Hour']== '3am EST') | (Daypart_Title['Start Hour']== '4am EST') | (Daypart_Title['Start Hour']== '5am EST'),
    (Daypart_Title['Start Hour']== '6am EST') | (Daypart_Title['Start Hour']== '7am EST') | (Daypart_Title['Start Hour']== '8am EST')| (Daypart_Title['Start Hour']== '9am EST'),
    (Daypart_Title['Start Hour']== '10am EST') | (Daypart_Title['Start Hour']== '11am EST') | (Daypart_Title['Start Hour']== '12pm EST'),
    (Daypart_Title['Start Hour']== '1pm EST') | (Daypart_Title['Start Hour']== '2pm EST') | (Daypart_Title['Start Hour']== '3pm EST'),
    (Daypart_Title['Start Hour']== '4pm EST') | (Daypart_Title['Start Hour']== '5pm EST') | (Daypart_Title['Start Hour']== '6pm EST') | (Daypart_Title['Start Hour']== '7pm EST'),
    (Daypart_Title['Start Hour']== '8pm EST') | (Daypart_Title['Start Hour']== '9pm EST') | (Daypart_Title['Start Hour']== '10pm EST'),
    (Daypart_Title['Start Hour']== '11pm EST') | (Daypart_Title['Start Hour']== '12am EST') | (Daypart_Title['Start Hour']== '1am EST')
]
    

Daypart = ['1. Overnight (2am-5am)','2. Morning (6am-9am)','3. Day1 (10am-12pm)','4. Day2 (1pm-3pm)',
           '5. Fringe (4pm-7pm)','6. Prime (8pm-10pm)','7. Late Night (11pm-1am)']

Daypart_Title['Daypart']= np.select(Condition, Daypart, default= np.nan)

In [25]:
Daypart_Title = Daypart_Title.loc[:,('Week','Start Hour','Daypart','Title','HOV','Minutes Viewed','Sessions','Session Duration')]

In [26]:
Daypart_Title.head(20)

,Week,Start Hour,Daypart,Title,HOV,Minutes Viewed,Sessions,Session Duration
0,2023-04-02/2023-03-27,11pm EST,Late Night (11pm-1am),10 Tokyo Warriors,657.77,39466.2,3150.0,12.528952
1,2023-04-02/2023-03-27,12am EST,Late Night (11pm-1am),10 Tokyo Warriors,25.91,1554.6,1192.0,1.304195
2,2023-04-02/2023-03-27,4pm EST,Fringe (4pm-7pm),10 Tokyo Warriors,0.01,0.6,2.0,0.300000
3,2023-04-02/2023-03-27,1am EST,Late Night (11pm-1am),Acrobunch - The Quest for Treasure,205.55,12333.0,92.0,134.054348
4,2023-04-02/2023-03-27,2am EST,Overnight (2am-5am),Acrobunch - The Quest for Treasure,806.94,48416.4,2317.0,20.896159
5,2023-04-02/2023-03-27,3am EST,Overnight (2am-5am),Acrobunch - The Quest for Treasure,206.65,12399.0,766.0,16.186684
6,2023-04-02/2023-03-27,6pm EST,Fringe (4pm-7pm),Acrobunch - The Quest for Treasure,0.01,0.6,3.0,0.200000
7,2023-04-02/2023-03-27,9pm EST,Prime (8pm-10pm),Appleseed,258.19,15491.4,790.0,19.609367
8,2023-04-02/2023-03-27,10pm EST,Prime (8pm-10pm),Appleseed,60.07,3604.2,1862.0,1.935661
9,2023-04-02/2023-03-27,5am EST,Overnight (2am-5am),Arcadia of My Youth: Infinite Course SSX,308.74,18524.4,792.0,23.389394


In [27]:
####

In [28]:
# Pivot Table

In [29]:
####

In [30]:
Daypart_Title_P = Daypart_Title.copy()

In [31]:
Daypart_Title_P

,Week,Start Hour,Daypart,Title,HOV,Minutes Viewed,Sessions,Session Duration
0,2023-04-02/2023-03-27,11pm EST,Late Night (11pm-1am),10 Tokyo Warriors,657.77,39466.2,3150.0,12.528952
1,2023-04-02/2023-03-27,12am EST,Late Night (11pm-1am),10 Tokyo Warriors,25.91,1554.6,1192.0,1.304195
2,2023-04-02/2023-03-27,4pm EST,Fringe (4pm-7pm),10 Tokyo Warriors,0.01,0.6,2.0,0.300000
3,2023-04-02/2023-03-27,1am EST,Late Night (11pm-1am),Acrobunch - The Quest for Treasure,205.55,12333.0,92.0,134.054348
4,2023-04-02/2023-03-27,2am EST,Overnight (2am-5am),Acrobunch - The Quest for Treasure,806.94,48416.4,2317.0,20.896159
...,...,...,...,...,...,...,...,...
197,2023-04-16/2023-04-10,12am EST,Late Night (11pm-1am),Yu-Gi-Oh!,8.89,533.4,296.0,1.802027
198,2023-04-16/2023-04-10,6pm EST,Fringe (4pm-7pm),Yu-Gi-Oh!,165.07,9904.2,249.0,39.775904
199,2023-04-16/2023-04-10,8pm EST,Prime (8pm-10pm),Yu-Gi-Oh! ARC-V,104.29,6257.4,154.0,40.632468
200,2023-04-16/2023-04-10,9pm EST,Prime (8pm-10pm),Yu-Gi-Oh! ARC-V,240.48,14428.8,1978.0,7.294641


# % of UV and SD (Last complete Month)

In [32]:
L_5=[]

for Week in Weeks_list:
    L_5 = L_5 + [Filepath_Roku_Viewership + Channel + '/' + Week + '/' + '% of Unique Viewers and Average Session Duration (By Title) (Last Complete Month).csv'] 
    #L_5 = L_5 + [Filepath_Roku_Viewership + Channel + '/' + Week + '/' + '%_of_unique_viewers_and_average_session_duration_(by_title)_(last_complete_month).csv']      

In [33]:
def Clean_UV_SD_Month(Data):
    UV_SD = pd.read_csv(Data)
    UV_SD = UV_SD.iloc[:,2:]
    UV_SD['Date'] = UV_SD.columns[1]
    UV_SD = UV_SD.iloc[1:,:]
    
    UV_SD.columns = ['Title','% of Viewers','AVG Session Duration (Hr)','Date']
    UV_SD['% of Viewers'] = UV_SD['% of Viewers'].str.replace('%','')
    UV_SD['% of Viewers'] = UV_SD['% of Viewers'].astype('float')
    UV_SD['AVG Session Duration (Hr)'] = UV_SD['AVG Session Duration (Hr)'].astype('float')
    UV_SD['AVG Session Duration (Min)'] = UV_SD['AVG Session Duration (Hr)']*60
    UV_SD['% of Viewers'] = UV_SD['% of Viewers']/100

    UV_SD = UV_SD.loc[:,('Date','Title','% of Viewers','AVG Session Duration (Min)')]
    return UV_SD.copy()

In [34]:
UV_SD_Title = pd.DataFrame()

for Data in L_5:
    UV_SD_Title = UV_SD_Title.append(Clean_UV_SD_Month(Data))

In [35]:
UV_SD_Title = UV_SD_Title.drop_duplicates(subset=['Date','Title'], keep='last')
UV_SD_Title = UV_SD_Title.sort_values('% of Viewers', ascending=False)

In [36]:
UV_SD_Title.head(10)

,Date,Title,% of Viewers,AVG Session Duration (Min)
1,2023-03-01,Tenjho Tenge,0.1042,5.4
2,2023-03-01,Hakugei: Legend of the Moby Dick,0.0930,8.4
3,2023-03-01,Virtua Fighter,0.0749,11.4
4,2023-03-01,Medabots,0.0735,10.2
5,2023-03-01,Ronin Warriors,0.0681,9.6
6,2023-03-01,City Hunter,0.0651,7.8
7,2023-03-01,GTO,0.0649,8.4
8,2023-03-01,The Twelve Kingdoms,0.0604,6.6
9,2023-03-01,Beelzebub,0.0603,12.0
10,2023-03-01,Street Fighter - The Animated,0.0559,9.6


# Export

In [37]:
output= pd.ExcelWriter(Channel + ' Roku_Linear_Report ' + Date + '.xlsx', engine='xlsxwriter')

View_CHN.to_excel(output, sheet_name= 'Channel Daily',index=False)
View_Title.to_excel(output, sheet_name= 'Title Daily',index=False)
Daypart_Channel.to_excel(output, sheet_name='Channel Daypart Weekly', index=False)
Daypart_Title.to_excel(output, sheet_name='Title Daypart Weekly', index=False)
UV_SD_Title.to_excel(output, sheet_name='Title UV_SD Monthly',index=False)

output.save()